**<h1>Jeden</h1>**

Niestety podczas przypisywania zdjęć do poszczególnych kategorii, odkryłeś coś strasznego. Marek Adamczyk od zawsze był fanatykiem cyfry 1, i miał on dziesiątki tysięcy jej zdjęć na swoim telefonie, a teraz nagle jest ich tylko jakoś 5000. Podejrzewasz, że gdy robił zdjęcia podczas swych przygód, musiał nadpisać swoją kolekcję. Wiesz, że jeśli zauważy co się stało, popadnie w rozpacz, a ty nigdy nie zobaczysz swoich pieniędzy. Korzystając z zdjęć dostępnych na jego telefonie, wygeneruj kolejne na tyle podobne, aby nie zauważył różnicy, i był szczęśliwy z swoją kolekcją zdjęć

Twoim zadaniem jest stworzenie systemu, który potrafi **generować nowe obrazy cyfry „1”**, bazując na danych MNIST. Obrazy te mają przypominać cyfrę „1” z MNIST, ale nie mogą być jej kopiami – muszą być **nowe**, wygenerowane na podstawie specyficznych reguł.

---

## Wyjaśnienie pojęć:

- **MNIST**: To zbiór danych zawierający ręcznie pisane cyfry (od 0 do 9) zapisane jako obrazy o rozdzielczości $28 \times 28$ pikseli. Każdy obraz to tak naprawdę wektor w przestrzeni $\mathbb{R}^{784}$, gdzie każda liczba odpowiada jasności konkretnego piksela.

- **Embedding ($\phi$)**:
  - Embedding to przekształcenie danych (tu: obrazów cyfry „1”) z ich oryginalnej postaci ($\mathbb{R}^{784}$) do bardziej zrozumiałej przestrzeni o niższym wymiarze (w tym przypadku $\mathbb{R}^3$).
  - Funkcja $\phi$ przypisuje każdemu obrazowi $x \in \mathbb{R}^{784}$ punkt w przestrzeni trójwymiarowej:

    $\phi : \mathbb{R}^{784} \to \mathbb{R}^3$
    
    Przykład: jeśli masz obraz $x_1$, to embedding $\phi(x_1)$ będzie punktem w przestrzeni $\mathbb{R}^3$, np. $(1.5, -2.3, 0.7)$.

- **Parametr ($\epsilon$)**:
  - To średnia odległość od każdego punktu w embeddingu $\phi(X^{1}_{mnist})$ do jego najbliższego sąsiada. Mówiąc prościej, $\epsilon$ określa, jak "gęsto" punkty są rozmieszczone w przestrzeni 3D:
  
    $$
    \Large \epsilon = \frac{1}{|X^{1}_{mnist}|} \sum_{x \in X^{1}_{mnist}} \min_{y \in X^{1}_{mnist} \setminus \{x\}} ||\phi(x) - \phi(y)||
    $$
---

## Kroki do wykonania

### 1. Przygotowanie danych
- Weź wyłącznie obrazy cyfry „1” z MNIST i oznacz ten zbiór jako $X^{1}_{mnist}$.
- Obrazy w tym zbiorze stanowią podstawę dla generacji nowych danych.

---

### 2. Przekształcenie danych do przestrzeni 3D (embedding $\phi$)
- Użyj algorytmu **t-SNE**, aby przekształcić obrazy $X^{1}_{mnist}$ z przestrzeni $\mathbb{R}^{784}$ (oryginalny format obrazów) do przestrzeni $\mathbb{R}^3$.
- Wyniki embeddingu to punkty $\phi(x)$ w przestrzeni 3D:

  $\phi(x) \in \mathbb{R}^3$

- Wyświetl chmurę punktów $\phi(X^{1}_{mnist})$ na wykresie 3D, aby zobaczyć ich rozmieszczenie.

---

### 3. Nauka odwzorowania z przestrzeni 3D na obrazy
- Zbuduj sieć neuronową, która nauczy się przekształcać punkty $\phi(x)$ z przestrzeni 3D z powrotem na odpowiadające im obrazy $x \in \mathbb{R}^{784}$:

  $\phi^{-1} : \mathbb{R}^3 \to \mathbb{R}^{784}$

- Sieć neuronowa ma przyjmować punkt $z \in \mathbb{R}^3$ i generować obraz $x \in \mathbb{R}^{784}$ (czyli obraz $28 \times 28$).
- **Ograniczenie**: Możesz używać tylko warstw gęstych (dense), żadnych warstw konwolucyjnych.

---

### 4. Generowanie nowych punktów w przestrzeni 3D
Aby wygenerować nowe obrazy, musisz najpierw stworzyć **losowe punkty $z_{\text{random}}$** w przestrzeni 3D. Masz dwa sposoby:

#### 4.1. Losowanie z rozkładu normalnego:
- Wybierz losowy punkt $\phi(x)$ z embeddingu $\phi(X^{1}_{mnist})$.
- Wygeneruj nowy punkt $z_{\text{random}}$ z rozkładu:

  $z_{\text{random}} \sim \mathcal{N}(\phi(x), \epsilon \cdot I)$
  
  gdzie:
  - $\mathcal{N}$ oznacza rozkład normalny,
  - $\phi(x)$ to wybrany punkt w embeddingu,
  - $\epsilon$ to średnia odległość do najbliższego sąsiada w $\phi(X^{1}_{mnist})$,
  - $I$ to macierz jednostkowa ($3 \times 3$).

#### 4.2. Losowanie z granic chmury punktów:
- Wybierz sześcian obejmujący całą chmurę punktów $\phi(X^{1}_{mnist})$, np. $[-20, 20] \times [-20, 20] \times [-20, 20]$.
- Generuj losowy punkt $z_{\text{random}}$ w tym sześcianie.
- Sprawdzaj odległość $d$ tego punktu do najbliższego punktu w $\phi(X^{1}_{mnist})$:
  $d = \min_{y \in \phi(X^{1}_{mnist})} ||z_{\text{random}} - y||$
- Jeśli $d \leq \epsilon$, akceptujesz punkt $z_{\text{random}}$. W przeciwnym razie go odrzucasz i próbujesz ponownie.

---

### 5. Generowanie obrazów
- Przekształć nowe punkty $z_{\text{random}}$ z przestrzeni 3D z powrotem na obrazy $x \in \mathbb{R}^{784}$, używając swojej sieci neuronowej $\phi^{-1}$.
- Wygenerowane obrazy powinny przypominać cyfry „1”, ale **nie mogą być identyczne** z obrazami w $X^{1}_{mnist}$.

---

### 6. Ocena jakości
- Oceń wygenerowane obrazy za pomocą klasyfikatora $k$-NN ($k = 3$):
  - Klasyfikator $k$-NN złożony z:
    - **Pierwszych 5,000 punktach z MNISTa** (klasy „1”) z zbioru treningowego,
    - **5,000 punktach wygenerowanych** przez Twój model $\phi^{-1}$.
  - **Accuracy (dokładność)** klasyfikatora należy sprawdzić na osobnym zbiorze testowym, który składa się z:
    - **Pierwszych 1,000 punktów z MNISTa** (klasy „1”) z zbioru testowego,
    - **1,000 punktów wygenerowanych** przez Twój model $\phi^{-1}$.

---
# <h2>Ocenanie</h2>

Ocena zależy od wartości F1-score w sposób liniowy, przy czym:

Wzór na obliczenie punktów (P):

$$ P = \frac{1-F1}{0.5} $$

Gdzie:
- `F1` to wartość F1-score modelu KNN (od 0 do 1).
- Jeśli `F1 <= 0.5`, to P = 1.

# <h2>Ograniczenia</h2>

1. Możesz skorzystać z gotowych implementacji t-SNE oraz KNN
2. Czas treningu i ewaluacji na GPU T4 w Google Colab powinien wynosić maksymalnie 15 minut.

<h2>Rozwiązanie</h2>

- W tym zadaniu musisz musisz dołączyć plik Zadanie.ipynb, który po włączeniu wykona wszystkie kroki polecenia, i w ostatniej komórce wyświetli accuracy.


Autor: Marek Adamczyk

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from sklearn.manifold import TSNE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random


In [ ]:
def set_seed(seed: int):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)


# Setting a seed for reproducibility
set_seed(42)
